In [ ]:
# Import necessary libraries
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from sklearn.metrics.pairwise import cosine_similarity
import io
from google.colab import files
from PIL import Image

# Load the pre-trained VGG16 model (exclude top layers) as feature extractor
model = VGG16(weights='imagenet', include_top=False, pooling='avg')

# Function to preprocess an image and extract its embedding
def get_embedding(img, target_size=(224, 224)):
    # Resize image to target size
    img = img.resize(target_size)
    # Convert image to array
    x = image.img_to_array(img)
    # Expand dimensions to match the model input
    x = np.expand_dims(x, axis=0)
    # Preprocess input for VGG16
    x = preprocess_input(x)
    # Get the image embedding
    embedding = model.predict(x)
    return embedding.flatten()

# Function to load an image from uploaded file bytes
def load_image(file_bytes):
    return Image.open(io.BytesIO(file_bytes)).convert('RGB')

# -------------------------------
# Step 1: Upload Product Catalog Images
# -------------------------------
print("Upload product catalog images (you can select multiple images):")
uploaded_catalog = files.upload()

# Store embeddings and images from the catalog
catalog_embeddings = []
catalog_images = []
catalog_filenames = []

for filename, filecontent in uploaded_catalog.items():
    img = load_image(filecontent)
    embedding = get_embedding(img)
    catalog_embeddings.append(embedding)
    catalog_images.append(img)
    catalog_filenames.append(filename)

catalog_embeddings = np.array(catalog_embeddings)
print("Uploaded {} catalog images.".format(len(catalog_images)))

# -------------------------------
# Step 2: Upload Query Image
# -------------------------------
print("Upload a query image for recommendation:")
uploaded_query = files.upload()

# Assume only one query image is uploaded
query_filename = list(uploaded_query.keys())[0]
query_image = load_image(uploaded_query[query_filename])
query_embedding = get_embedding(query_image)

# -------------------------------
# Step 3: Compute Similarity and Recommend
# -------------------------------
# Calculate cosine similarity between the query image and each catalog image
similarities = cosine_similarity([query_embedding], catalog_embeddings)[0]

# Get top 3 recommendations (highest similarity scores)
top_k = 3
top_indices = np.argsort(similarities)[::-1][:top_k]

print("Top {} recommended images:".format(top_k))
for idx in top_indices:
    print("Filename: {}, Similarity: {:.2f}".format(catalog_filenames[idx], similarities[idx]))

# -------------------------------
# Step 4: Display Results
# -------------------------------
plt.figure(figsize=(12, 4))

# Show the query image
plt.subplot(1, top_k + 1, 1)
plt.imshow(query_image)
plt.title("Query Image")
plt.axis('off')

# Show the top recommended images with similarity scores
for i, idx in enumerate(top_indices):
    plt.subplot(1, top_k + 1, i + 2)
    plt.imshow(catalog_images[idx])
    plt.title("Sim: {:.2f}".format(similarities[idx]))
    plt.axis('off')

plt.show()
